# Tallying unique items
Natalia Vélez, June 2020

Some items have special modifiers attached to the object ID:

```
483.81 -5123 -1390 2742u3 2276909
…
511.16 -5123 -1392 2742u2 2276909
…
516.87 -5123 -1392 2742u1 2276905
…
522.20 -5123 -1392 2742u0 2276914
```

That exact ID “2742u3” doesn’t match any objects, but it’s possible that “u[0-9]” is a counter on how many uses are remaining in an object with multiple uses. Object 2742 is a carrot pile, which has up to 5 uses. This log could show that multiple players went up to this carrot pile and ate from it.

How many modifiers are there, and what do they mean?

In [1]:
import os, re, glob, datetime
from os.path import join as opj
import pandas as pd
import numpy as np
from tqdm import notebook

Maplog files:

In [2]:
map_files = glob.glob('outputs/maplog/maplog*.tsv')
map_files.sort()

print(*map_files[:5], sep='\n')

outputs/maplog/maplog_release-284_start-1573895672.tsv
outputs/maplog/maplog_release-284_start-1573982073.tsv
outputs/maplog/maplog_release-284_start-1574068473.tsv
outputs/maplog/maplog_release-285_start-1574102503.tsv
outputs/maplog/maplog_release-287_start-1574151678.tsv


Helper: Process individual file

In [3]:
def find_unique(file):

    # Trim invalid objects
    map_df = pd.read_csv(file, sep='\t')
    map_df = map_df[map_df.player_id > 0] # No player attached to event
    map_df = map_df[map_df.object_id != '0'] # Interact w/ empty square 
    
    # Clean up modifiers
    map_df['object_id'] = map_df.object_id.str.replace(r'^(f)|([u-v][0-9]+$)', '')
    
    # Convert to number
    map_df['object_id'] = map_df['object_id'].astype(np.int64)
    map_df['object_id'] = np.where(map_df['object_id'] > 5000, 9999, map_df['object_id'])

    # Unique items
    obj_list = map_df.object_id.tolist()
    unique_obj = np.unique(obj_list).tolist()
    unique_obj.sort()
    
    return unique_obj

Iterate over all files:

In [4]:
unique_object_list = []
for f in notebook.tqdm(map_files):
    unique_object_list.append(find_unique(f))

Combine everything into one big list of unique items across all files;

In [5]:
unique_objects_flat = sum(unique_object_list,[])
unique_objects_flat = np.array(unique_objects_flat)

unique_objects = np.unique(unique_objects_flat)
print('Found %i unique objects' % len(unique_objects))
print(*unique_objects[:50], sep='\t')

Found 2805 unique objects
30	31	32	33	34	35	39	40	45	48	49	53	54	55	57	58	59	61	62	63	64	65	66	67	68	69	70	71	72	73	74	75	77	78	79	80	82	83	85	86	87	92	96	99	100	101	103	104	105	106


Save this to file (we'll need it when we generate activity matrices!):

In [6]:
out_file = 'outputs/activity_features.txt'
with open(out_file, 'w') as handle:
    for obj in unique_objects:
        handle.write('%s\n' % obj)